<a href="https://colab.research.google.com/github/SKGHD/Handy/blob/master/HandyCLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Paste this code in your Console to stop Colab from disconnecting</h3>

```
function ClickConnect() {
  console.log('Working')
  document
    .querySelector('#top-toolbar > colab-connect-button')
    .shadowRoot.querySelector('#connect')
    .click()
}
setInterval(ClickConnect, 60000)
```







In [ ]:
#@title <Center>Mount Google Drive </Center>
#@markdown <Center> <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" alt="Icon">

from google.colab import drive
drive.mount('/content/drive')

#Run These first

In [ ]:
#@title <---Install HandBrake and rClone
!mkdir -p /content/temp/HandbrakeTemp
!mkdir /content/drives/
!mkdir -p /root/.config/rclone

!apt-get install handbrake-cli
!curl https://rclone.org/install.sh | sudo bash

from os import system
from time import sleep 
def clear():
  system('clear')

In [ ]:
#@title <-----Upload Rclone .config file
def moveConfig():
  !mv /content/rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

# <center>Rclone Mount/Unmount/Copy </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode full --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
 
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
mode = "copy" #@param ["copy", "check"]
dry_run = False #@param {type:"boolean"}
#@markdown ---
Email_notification = True #@param {type:"boolean"}
logs = False #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    if dry_run==True:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run 
    elif dry_run==False:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
    #Send only notification
    import smtplib 
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your Task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    if dry_run==True:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run --log-file logs.txt 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run --log-file logs.txt 
    elif dry_run==False:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --log-file logs.txt 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --log-file logs.txt 
 
 
    #Sending email notification with logs
    import smtplib 
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 
 
    fromaddr = emailID
    toaddr = Receiver_ID
 
     
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your Task has Completed! Visit link to view : https://colab.research.google.com/github/SKGHD/Handy/blob/master/HandyCLI.ipynb "

    msg.attach(MIMEText(body, 'plain'))
    filename = "logs.txt"
    attachment = open("/content/logs.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text) 
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    if dry_run==True:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run 
    elif dry_run==False:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
checkEmail()

# <center>***HandBrake*** </center> 
#<Center> <img src="https://handbrake.fr/img/logo.png" width="40"></center>

In [ ]:
#@title <center> Handbrake Configuration </center>
#@markdown ---
import smtplib 
SOURCE = "" #@param {type:"string"}
DESTINATION = "" #@param {type:"string"}
SAVE_FILE_NAME = "" #@param {type:"string"}
#rTemp="/content/temp/vfs/"+NAME
hTemp="/content/temp/HandbrakeTemp/"
hTemp +=SAVE_FILE_NAME
#!rclone copy "$SOURCE" "$rTemp" 
FORMAT = "mp4" #@param ["mp4", "mkv"]
RESOLUTION = "480p" #@param ["Manual Input", "480p", "720p", "1080p"]

Encoder_Preset = "ultrafast" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 22 #@param {type:"slider", min:10, max:30, step:1}
Additional_Flags = "" #@param {type:"string"}
#@markdown ---
Email_Alert = True #@param {type:"boolean"}
MESSAGE = "HandBrake has finished working" #@param ["HandBrake has finished working"] {allow-input: true}
SENDER_ID ="" #@param {type:"string"}
SENDER_PASS="" #@param {type:"string"}
RECEIVER_ID="" #@param {type:"string"}

def res():
  if RESOLUTION == "Manual Input":
    w=input("Enter Width:")
    h=input("Enter Height:")
  elif RESOLUTION == "480p":
   w, h = "854" , "480"
  elif RESOLUTION == "720p":
   w, h = "1280" , "720"
  elif RESOLUTION=="1080p":
   w, h = "1920" , "1080"
  def addFlags():
    flags = " --encoder x265  --all-audio --all-subtitles --cfr --optimize "
    flags += "--quality="+str(CQ)+" "
    flags += "--width="+str(w)+" "
    flags += "--height="+str(h)+" "
    flags += "--format="+str(FORMAT)+" "
    flags += "--encoder-preset="+str(Encoder_Preset)+" "
    if Additional_Flags != "":
      flags += str(Additional_Flags)   
    !HandBrakeCLI -i $SOURCE -o $hTemp $flags
  addFlags()
res()
print("rClone will now move the transcoded file to :",remote_name,"/Handbrake/" ) 
!rclone move $hTemp $DESTINATION -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
if Email_Alert == True:
  s = smtplib.SMTP('smtp.gmail.com', 587) 
  s.starttls() 
  s.login(SENDER_ID, SENDER_PASS)
  s.sendmail(SENDER_ID, RECEIVER_ID, MESSAGE)
  print("Email Alert Sent!")  
  s.quit()

# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Diamond-caution.svg/780px-Diamond-caution.svg.png" width=30> TESTING AREA (Unfinished) <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Diamond-caution.svg/780px-Diamond-caution.svg.png" width=30>


# <center>Rclone Mount/Unmount/Copy </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
 
#@markdown <center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "" #@param {type:"string"}
destination = "" #@param {type:"string"}
mode = "check" #@param ["copy", "check"]
dry_run = False #@param {type:"boolean"}
#@markdown ---
Email_notification = True #@param {type:"boolean"}
logs = True #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
def runrClone():
    if dry_run==True:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 --dry-run 
    elif dry_run==False:
      if mode=="copy":
        !rclone "$mode" "$source" "$destination" -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
      elif mode=="check":
        !rclone "$mode" "$source" "$destination" --one-way -P --user-agent "Mozilla" --stats-one-line --stats=5s -v  --transfers 20 --checkers 20 --tpslimit 95 --tpslimit-burst 40 
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    runrClone()
    #Send only notification
    import smtplib 
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    runrClone()
    #Sending email notification with logs
    import smtplib 
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your rClone Task has Completed! Visit link to view : https://colab.research.google.com/drive/1_j45gMIPVOEl5FBlHmu9exsZMYvIyMdx#scrollTo=yoio4Xr4Ogsl&line=48&uniqifier=1"

    msg.attach(MIMEText(body, 'plain'))
    filename = "logs.txt"
    attachment = open("/content/logs.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent!")  
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runrClone()
checkEmail()